# Dendrite Model

1. Create a passive cable model using th eneural parameters described in the McIntyre, Richardson, and Grill Paper provided in the reading. Your model shall consist of three dendrites connected to a cell body. The cell body is assumed to be a sphere with diameter 50 $\mu$m, the dendrites are cylinders with the following diameters and lengths

  D1: 
    
    * L=1mm
    * d = 10 $\mu$m
    
  D2: 
    
    * L = 1mm
    * d = 20 $\mu$m
    
  D3:
    
    * L=300 $\mu$m
    * d = 20 $\mu$m
    
  Show the change in potential at the cell body as a function of an injected square pulse of cathodal (depolarizing) current under the conditions listed below. After all of these modeling exercises, describe the influence of the diameter and length of the dendrites on processing of synaptic information.
    
   * Same, single pulse input in the middle of each of the respective dendrites.
   * Same, single pulse input equidistant from the cell body for each dendrite.
   * Multiple input pulses of current at equidistance from the cell body for eachdendrite for several frequencies.
    
2. Add an active axon to your model. Double the sodium channel density in the initial axon hillock segment of the axon. Repeat the scenarios above and describe the effects on the axon. Find input pulses that drive the axon to fire an action potential. The axon should be 20 $\mu$m in diameter, is myelinated, and should consist of 20 nodes after the initial segment.
        
3. Rather than injecting a current internally at the node of the dendrite, add a synaptic current across the membrane. Use a single tau model and set tau to different values ranging from 50 $\mu$s and th emaximum synaptic conductance initially equal tot he sodium conductance and then varied higher and lower. Explore spatial and temporal summation using your model. Vary the frequency of input pulses that are individually too small to cause axon excitement.

4. Change the diameters of the cell body from 30 $\mu$m to 100 $\mu$m with active axons changing in proportion from 8 $\mu$m to 20 $\mu$m. Describe the impact of the size change on the dendritic input and on the axon excitation. For a fixed size, examine the effect of increasing intermodal spacing from 100*D to 500*D on action potential propagation.

## Math

### Nernst potential

The nernst potential is governed by the equation:

$$V_{ion} = \frac{RT}{zF} \ln \left( \frac{[ion]_{out}}{[ion]_{in}} \right)$$

Where

$$V_{ion} = V_{in} - V_{out}$$

$R$ is the gas constant 8.314 J/K/Mol

$T$ is temperature in kelvin

$z$ is the valence of the ion

$F$ is Faradays constant 96,485 C/mol

Given $T$ is 37 C, then 

$$V_{ion} = \frac{26.7}{z} \ln \left( \frac{[ion]_{out}}{[ion]_{in}} \right)$$
$$V_{ion} = \frac{61.5}{z} \log \left( \frac{[ion]_{out}}{[ion]_{in}} \right)$$

### GHK equation

When multiple ions are involved, the following relation occurs:

$$V_{GHK} = \frac{RT}{F} \times \ln \left[ \frac{\sum_{+ions} P_{ion} [ion^+]_{out} + \sum_{-ions} P_{ion} [ion^-]_{in}}{\sum_{+ions}P_{ion}[ion^+]_{in} + \sum_{-ions} P_{ion}[ion^-]_{out}}\right]$$

where $P_{ion}$ is the membrane permiability of that particular ion.

### Passive Cable equation

The passive cable equation is derived by assuming the dendrite is a cylinder. It assumes that all the ion channels on the membrane can be represented by a single resistor. $\Delta x$ from the integration actually gets factored out and becomes a partial derivative term for the external voltage.


$$V_m + \tau_m \frac{\partial V_m}{\partial t} - \lambda^2 \frac{\partial^2 V_m}{\partial x^2} - i_{inj} = \lambda^2 \frac{\partial^2 V_e}{\partial x^2}$$

$$\tau_m = r_m c_m$$
$$\lambda = \frac{r_m}{r_a}$$

So what does the cable equation represent?

It represents the relationship between the membrane voltage, the external voltage, and any injected voltage.

If the equation is normally derived, the $i_{inj}$ term is actually $\frac{i_{inj} r_m}{\Delta x}$, but you can ignore the $\Delta x$ term if you assume the current is injected from a point, and $r_m$ can be ignored if you assume that it is injected directly inside.

So for the model, what are the inputs, and what are the outputs?

The input is $i_{inj}$ and the outputs are actually $i_a+$ and $i_a-$ using these and superposition, we'll be able to chain them to other nodes.

## Conversion to code:

The compartment model discretizes the continuous cable model by discrete steps for a compartment $j$, and assuming that the external voltage, $V_e$ is zero, we get the following equation:

$$\frac{V_j}{R_m} + C_m \frac{d V_j}{d t} -  \frac{V_{j+1} - 2V_j + V_{j-1}}{r_a \Delta x^2} - i_{inj} = 0$$

Next you can generalize that the voltages of each compartment are related by different $r_a$s.

$$\frac{V_j}{R_m} + C_m \frac{d V_j}{d t} +  \frac{V_j - V_{j+1}}{0.5\cdot(R_{a,j}+R_{a,j+1})} +\frac{V_j - V_{j-1}}{0.5\cdot(R_{a,j}+R_{a,j-1})} - i_{inj} = 0$$

Where
$$R_m =\frac{r_m}{\Delta x}$$
$$C_m =c_m\Delta x$$
$$R_a =r_a\Delta x$$

You can rearrange the equation to get the following:

$$C_m \frac{d V_j}{d t} = -\left(\frac{1}{r_m \Delta x} + \frac{1}{0.5 \cdot (r_{a,j} + r_{a,j+1})} + \frac{1}{0.5 \cdot (r_{a,j} + r_{a,j-1})} \right)V_j + \left(\frac{1}{0.5\cdot(r_{a,j}+r_{a,j+1})}\right)V_{j+1}+ \left(\frac{1}{0.5\cdot(r_{a,j}+r_{a,j-1})}\right)V_{j-1} $$

Which can be rearranged into the following matrix:

$$
\begin{bmatrix}
\frac{d V_{j}}{dt} \\
\frac{d V_{j+1}}{dt} \\
\frac{d V_{j+2}}{dt} \\
\frac{d V_{j+3}}{dt} \\
\vdots
\end{bmatrix} 
= 
A\begin{bmatrix}
V_j \\
V_{j+1} \\
V_{j+2} \\
V_{j+3} \\
V_{j+4}
\end{bmatrix}
+
\begin{bmatrix}
i_{inj,j} \\
i_{inj,j+1} \\
i_{inj,j+2} \\
i_{inj,j+3} \\
i_{inj,j+4}
\end{bmatrix}
$$

$$d\overrightarrow{V} = A\overrightarrow{V} + \overrightarrow{I}$$

Where the matrix A has coefficients which represent the resistances connecting the nodes with the adjacent nodes. Assuming all voltages start from zero, we can integrate the system over time to produce the voltage output.

In [2]:
using Plots

In [34]:
#Initialize compartment, figure out scoping of named parameters
module NeuronSim

# TYPE DEFINITIONS
type Compartment
    parent_::Int
    children_::Vector{Int}
    id_::Int

    len_::Float64

    r_m_::Float64
    R_m_::Float64

    c_m_::Float64
    C_m_::Float64

    r_a_::Float64
    R_a_::Float64

    V_m_::Float64
end

type NeuronModel
    nodes::Vector{Compartment}
end

# AUXILIARY FUNCTIONS

# adds a child compartment to the parent
function AddRoot!(model::NeuronModel, root::Compartment)
    AddNodeById!(model, 0, root)
end

function AddNode!(model::NeuronModel, parent::Compartment, child::Compartment)
    AddNodeById!(model, parent.id_, child)
end

function AddNodeById!(model::NeuronModel, parent_id::Int, child::Compartment)
    push!(model.nodes, child)
    child.id_ = length(model.nodes)
    if(parent_id > 0)
        child.parent_ = parent_id
        push!(model.nodes[parent_id].children_, child.id_)
    end
end

# initializes and creates a model
function CreateModel()
    return NeuronModel([])
end

function CreateCompartment(;length::Float64=0.0, 
        r_m::Float64=0.0, 
        c_m::Float64=0.0, 
        r_a::Float64=0.0, 
        V_initial::Float64=0.0)
    
    comp = Compartment(
        0,
        [],
        0,
        length,
        r_m,
        r_m/length,
        c_m,
        c_m*length,
        r_a,
        r_a*length,
        V_initial)
end
end

ns = NeuronSim
model = ns.CreateModel()
root = ns.CreateCompartment(length=10.0, r_m =1.0, c_m=1.0, r_a=1.0, V_initial=1.0)

ns.AddRoot!(model, root)


for i in 1:5
    compartment = ns.CreateCompartment(length=20.0*i, r_m =1.0, c_m=1.0, r_a=1.0, V_initial=1.0)
    ns.AddNode!(model, root, compartment)
end

for i in 1:5
    compartment = ns.CreateCompartment(length=20.0*i, r_m =1.0, c_m=1.0, r_a=1.0, V_initial=1.0)
    ns.AddNodeById!(model, 5, compartment)
end

println(compartment)

map(println, model.nodes)


NeuronSim.Compartment(5,Int64[],11,100.0,1.0,0.01,1.0,100.0,1.0,100.0,1.0)


NeuronSim.Compartment(0,[2,3,4,5,6],1,10.0,1.0,0.1,1.0,10.0,1.0,10.0,1.0)
NeuronSim.Compartment(1,Int64[],2,20.0,1.0,0.05,1.0,20.0,1.0,20.0,1.0)
NeuronSim.Compartment(1,Int64[],3,40.0,1.0,0.025,1.0,40.0,1.0,40.0,1.0)
NeuronSim.Compartment(1,Int64[],4,60.0,1.0,0.016666666666666666,1.0,60.0,1.0,60.0,1.0)
NeuronSim.Compartment(1,[7,8,9,10,11],5,80.0,1.0,0.0125,1.0,80.0,1.0,80.0,1.0)
NeuronSim.Compartment(1,Int64[],6,100.0,1.0,0.01,1.0,100.0,1.0,100.0,1.0)
NeuronSim.Compartment(5,Int64[],7,20.0,1.0,0.05,1.0,20.0,1.0,20.0,1.0)
NeuronSim.Compartment(5,Int64[],8,40.0,1.0,0.025,1.0,40.0,1.0,40.0,1.0)
NeuronSim.Compartment(5,Int64[],9,60.0,1.0,0.016666666666666666,1.0,60.0,1.0,60.0,1.0)
NeuronSim.Compartment(5,Int64[],10,80.0,1.0,0.0125,1.0,80.0,1.0,80.0,1.0)
NeuronSim.Compartment(5,Int64[],11,100.0,1.0,0.01,1.0,100.0,1.0,100.0,1.0)


11-element Array{Void,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing